# Test torch geometric

In [1]:
import os 
import networkx as nx 
import pickle 
import torch_geometric 

data_dir = r'Y:\coskun-lab\Thomas\23_PLA_revision\data\9PPI\graphs\raw'

In [2]:
for f in os.listdir(data_dir):
    with open(os.path.join(data_dir, f), 'rb') as file:
        G = pickle.load(file)
    data = torch_geometric.utils.from_networkx(G)
    break

c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\torch_geometric\utils\convert.py:192: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  data[key] = torch.tensor(value)


In [3]:
import wandb

# Loop code

In [4]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import tifffile as tiff
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [5]:
import torch_geometric.utils
import torch

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [9]:
import lightning.pytorch as pl
import PPIGraph

# Load dataset

In [10]:
from torch_geometric.loader import DataLoader

# Define condition mapping
condition_mapping = {'HCC827Ctrl': 0, 'HCC827Osim': 1}

# Load graph dataset and process if neede
graph_path = data_dir / 'OCT Cell Culture' / 'Whole' / 'graphs' 

dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [11]:
# Print some information on the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
for step, data in enumerate(train_loader):

    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Dataset: GraphDataset(2064):
Number of graphs: 2064
Number of features: 5
Number of classes: 2
Train set: 992, test set: 825, val set: 247
Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 10064], pos=[1996, 2], labels=[1996, 5], nuclei=[1996], weight=[10064], condition=[32], fov=[32], id=[32], train_mask=[1996], test_mask=[1996], edge_attr=[10064, 2], x=[1996, 5], y=[32], edge_weight=[10064], name=[32], batch=[1996], ptr=[33])



# K-fold on filter dataset

In [12]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [13]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [14]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 70

graph_path = data_dir / 'OCT Cell Culture' / 'Whole' / 'graphs' 
dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [15]:
len(dataset_filtered)

1522

In [16]:
dataset_name = '5PPI'
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' /'OCT Cell Culture' / 'Whole' /"saved_models" / dataset_name / f"Graph_GNNs_{condition}" 
project_name = f'PLA_01102024_{dataset_name}_{condition}'

In [17]:
AVAIL_GPUS = [1]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 2
HIDDEN_CHANNELS = 16
pools = ['mean', 'max', 'sum', 'attention', 'attention2']
num_layers = [2,3,4]
hiddens = [16, 32, 64]

epochs = 50
models = ['GAT']

Global seed set to 42


In [18]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
    train_subset = dataset_filtered.index_select(train_ids.tolist())
    val_subset = dataset_filtered.index_select(valid_ids.tolist())
    for NUM_LAYERS, HIDDEN_CHANNELS, pool, model in list(itertools.product(*[num_layers, hiddens, pools, models])):
            # Path to the folder where the pretrained models are saved
        CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

        # Skip already trained kfold and pool
        checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
        if checkpoint.exists():
            print(checkpoint)
            continue

        # Run training
        run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}', 
                        group=f'{model}_{pool}', 
                        # mode="disabled"
                        )
        PPIGraph.train_graph_classifier_kfold(model, 
                                                train_subset, 
                                                val_subset, 
                                                dataset, 
                                                CHECKPOINT_PATH, 
                                                AVAIL_GPUS, 
                                                in_channels=5,
                                                hidden_channels=HIDDEN_CHANNELS, 
                                                out_channels = HIDDEN_CHANNELS,
                                                num_layers=NUM_LAYERS, 
                                                epochs=epochs,
                                                embedding=False,
                                                batch_size=256, 
                                                graph_pooling=pool)
        run.finish()


y:\coskun-lab\Thomas\23_PLA_revision\data\OCT Cell Culture\Whole\saved_models\5PPI\Graph_GNNs_Kfold\GAT_2_16_onehot_0\mean\GraphLevelGAT\GraphLevelGAT.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT Cell Culture\Whole\saved_models\5PPI\Graph_GNNs_Kfold\GAT_2_16_onehot_0\max\GraphLevelGAT\GraphLevelGAT.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT Cell Culture\Whole\saved_models\5PPI\Graph_GNNs_Kfold\GAT_2_16_onehot_0\sum\GraphLevelGAT\GraphLevelGAT.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT Cell Culture\Whole\saved_models\5PPI\Graph_GNNs_Kfold\GAT_2_16_onehot_0\attention\GraphLevelGAT\GraphLevelGAT.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT Cell Culture\Whole\saved_models\5PPI\Graph_GNNs_Kfold\GAT_2_16_onehot_0\attention2\GraphLevelGAT\GraphLevelGAT.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT Cell Culture\Whole\saved_models\5PPI\Graph_GNNs_Kfold\GAT_2_32_onehot_0\mean\GraphLevelGAT\GraphLevelGAT.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT Cell Cult

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42


Using GAT


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model       | GAT              | 69.3 K
2 | head        | Sequential       | 2.1 K 
3 | loss_module | CrossEntropyLoss | 0     
4 | train_acc   | BinaryAccuracy   | 0     
5 | train_auroc | BinaryAUROC      | 0     
6 | train_f1    | BinaryF1Score    | 0     
7 | valid_acc   | BinaryAccuracy   | 0     
8 | valid_auroc | BinaryAUROC      | 0     
9 | valid_f1    | BinaryF1Score    | 0     
-------------------------------------------------
71.5 K    Trainable params
0         Non-trainable params
71.5 K    Total params
0.286     Total estimated model params size (MB)
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shu

...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▃▅▇▆▆▆▇▇▇▇▇▇▆▇▇█▇█▇▇▇█▇▇▇▇▇▇▇▇▇███▇█▇▇█
train_auc,▁▄▆▇▇▇▇▇▇██▇▇▇█████████▇███████████▇██▇█
train_f1,▂▁▅▇▇▇▇▇▇▇███▇▇███▇█▇██████▇▇██▇██████▇█
train_loss_epoch,█▆▅▂▃▂▂▂▂▂▂▂▃▂▂▁▁▂▂▂▂▂▁▂▁▂▂▂▂▁▂▁▁▁▁▂▁▂▃▁
train_loss_step,█▂▂▃▄▁▃▅▁▃▄▅▃▂▄▅▁▃▁▃▄▂▅█▆
trainer/global_step,▁▁▁▁▂▁▂▁▂▂▃▃▃▃▄▄▂▄▂▄▂▅▃▅▃▆▆▆▃▃▇▇▇▃▇▄█▄█▄
val_acc,▁▁▃▆█▅▅▃▄▄▄▆▅▆▆▆▇▇█▆▄▃▄▂▂▃▄▆▇█▆█▅▅▅▅▂▄▄▇
val_auc,▃▄▁▁▅▅▅▆▆▇▆▆▇▇▅▆▆▇▆▆███▆▆▇▇▅▅▆▅▆▇██▇▆▆▆▇
val_f1,▁▁▄▆███▇███████████████▇▇▇██████████▇███


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model       | GAT              | 86.7 K
2  | head        | Sequential       | 2.1 K 
3  | loss_module | CrossEntropyLoss | 0     
4  | train_acc   | BinaryAccuracy   | 0     
5  | train_auroc | BinaryAUROC      | 0     
6  | train_f1    | BinaryF1Score    | 0     
7  | valid_acc   | BinaryAccuracy   | 0     
8  | valid_auroc | BinaryAUROC      | 0     
9  | valid_f1    | BinaryF1Score    | 0     
10 | pool        | Attention_module | 4.2 K 
--------------------------------------------------
93.1 K    Trainable params
0         Non-trainable params
93.1 K    Total params
0.372     Total estimated model para

Using GAT


`Trainer.fit` stopped: `max_epochs=50` reached.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▃▅▆▆▇▇▇▇▆▇▇▇▇▇▇▇▇█▇█▆███▇█▇███▇██▇▇▇▇▇▇
train_auc,▁▅▆▇▇▇█▇█▇████▇▇█████▇███▇█████████████▇
train_f1,▂▁▇▇▆▇▇▇▇▇▇█▆█▇▆▇▇█▇█▇▇██▇█▇██████▇▇▇█▇▇
train_loss_epoch,█▆▄▃▃▂▂▂▂▂▁▁▂▃▂▂▂▂▂▂▂▁▂▂▁▂▁▂▁▂▂▁▂▁▁▂▁▂▂▂
train_loss_step,█▅▃▄▆▃▁▇▂▆▃▄▃▂▅▂▂▁▃▁▄▅▃▂▁
trainer/global_step,▁▁▁▁▂▁▂▁▂▂▃▃▃▃▄▄▂▄▂▄▂▅▃▅▃▆▆▆▃▃▇▇▇▃▇▄█▄█▄
val_acc,▂▂▄▆▇▅▃▃▅▃▃▃▆▃▇▄▆▂▂▅▄▇▇█▆▆▃▃▂▂▃▁▂▄▅▅▇▇▅▅
val_auc,▄▂▃▁▄▄▁▂▆▆▆▄▄▅▆▄▇▆▆█▅▆▃▄▄▆▅▄▄▅▄▅▄▅▅▄▅▆▇▅
val_f1,▁▁▄▆███▇███▇█████▇▇███████▇▇▇▇▇▇▇███████
